**LIBRERIAS**

In [1]:
import pandas as pd
import pyarrow
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

True

**CONEXIONES**

In [2]:
try:
    # Conexión a SalesSystem (Facturación) - Comentada pero usando variables de entorno
    """
    salessystem_url = f"{os.getenv('DB_SALESSYSTEM_DIALECT')}://{os.getenv('DB_SALESSYSTEM_USER')}:{os.getenv('DB_SALESSYSTEM_PASSWORD')}@{os.getenv('DB_SALESSYSTEM_HOST')}:{os.getenv('DB_SALESSYSTEM_PORT')}/{os.getenv('DB_SALESSYSTEM_NAME')}"
    salessystem = create_engine(salessystem_url)
    """
    
    # Conexión a Warehouse (Contabilidad)
    warehouse_url = f"{os.getenv('DB_WAREHOUSE_DIALECT')}://{os.getenv('DB_WAREHOUSE_USER')}:{os.getenv('DB_WAREHOUSE_PASSWORD')}@{os.getenv('DB_WAREHOUSE_HOST')}:{os.getenv('DB_WAREHOUSE_PORT')}/{os.getenv('DB_WAREHOUSE_NAME')}"
    warehouse = create_engine(warehouse_url)
    
    # Validar la conexión
    with warehouse.connect() as conn_warehouse:
        pass
        
except Exception as e:
    print(f"Error al conectar a la base de datos: {str(e)}")
    raise

**VARIABLES**  

In [3]:
def get_mes_anterior(meses):
    fecha_actual = datetime.now()
    mes_anterior = fecha_actual - relativedelta(months=meses)
    return int(mes_anterior.strftime('%Y%m'))
periodo_cerrado = get_mes_anterior(1)
periodo_corriente = get_mes_anterior(0)
periodo_seleccionado = 202410

**CONSULTAS SQL**

In [4]:
priv_entities=pd.read_sql("SELECT * FROM priv.entities ORDER BY ruc ASC", warehouse)
acc_9=pd.read_sql("SELECT * FROM acc._9", warehouse)

**EMPRESAS ORDENADAS SEGUN ULTIMO DIGITO**

In [5]:
entities_1 = priv_entities[priv_entities['activo'] == True].assign(
    ultimo_digito_ruc=lambda x: x['ruc'] % 10
).sort_values(by='ultimo_digito_ruc').drop(columns='ultimo_digito_ruc')
entities_1=entities_1[['nombre_razon', 'ruc', 'usuario_sol', 'clave_sol', 'observaciones']]

In [6]:
entities_1

,nombre_razon,ruc,usuario_sol,clave_sol,observaciones
90,NOVATEX GLOBAL S.A.C,20603078200,LAMPTUDE,Cesar2022,None
120,JMV INVESTMENTS E.I.R.L,20611097400,10813326,Jorge2703,None
58,GEOSSTRATOS GEOTECHNICAL SOLUTIONS S.A.C.,20522580491,QUITORKN,Fabio2022,None
33,JACQUELINE CORDOVA,10469541661,BILENTRA,opertatio,None
98,IML NEGOCIACIONES E.I.R.L.,20605380451,75950584,Imlsaave03,None
117,ROMARK PROJECTOS S.A.C.,20610704281,41993294,13092023Ms,None
114,CLARDENT,20609943611,47251373,Clardent3611,None
53,MINIER DE OLEO DICKERSON MARINO,15609947851,OFFOROAR,iambustek,None
116,SONIC SERVICIOS VARIOS E.I.R.L,20610428101,47205286,Cabezaroper1,None
130,CONSULTORIA Y ASESORIA CELIZ ASOCIADOS E.I.R.L.,20614580951,EXPETROF,arknozeie,None


**DECLARACIONES PENDIENTES MES EN CURSO**

In [7]:
# Filtro de periodo
acc_9_filtrado = acc_9[(acc_9['periodo_tributario'] == periodo_cerrado)]
# Filtro de no declarados
acc_9_filtrado = acc_9_filtrado[acc_9_filtrado['numero_orden'].isna()]
# Filtro de columnas
acc_9_filtrado = acc_9_filtrado[['ruc', 'periodo_tributario', 'fecha_presentacion','_100','_107','_301','_145']]
# Realizar un join entre priv_entities y acc_9_filtrado
resultado_join = entities_1.merge(acc_9_filtrado, on='ruc', how='left')
resultado_sin_coincidencias = resultado_join[resultado_join['periodo_tributario'].notna()]

In [8]:
resultado_sin_coincidencias

,nombre_razon,ruc,usuario_sol,clave_sol,observaciones,periodo_tributario,fecha_presentacion,_100,_107,_301,_145


In [1]:
#NUEVA ESTRUCTURA DE REPORTE PARA PAGO DE PLANILLAS DEL MES
'''	join	join	join		concat1	concat1	if tipo_contrato A TIEMPO PARCIAL horas trabajadas 120, default 240				
ide.ruc	entities.usuario_sol	entities.clave_sol	entities.alias	ide.numero_documento	ide.nombre	ide.apellido_paterno	tra.tipo_contrato	ssc.salud	ssc.pension	comisiones_afp.sobre_flujo	comisiones_afp.obligatorio
tipo_documento
numero_documento
apellido_paterno
apellido_materno
nombres
situacion
tipo_trabajador
tipo_regimen_salud
fecha_inicio_salud
EPS/Serv. Propio
tipo_regimen_pension
fecha_inicio_pension
CUSPP
sctr_salud_ninguno
sctr_salud_essalud
scrt_salud_eps
scrt_pension_ninguno
sctr_pension_onp
sctr_pension_privado
rentas_lir
evit_doble_imp
'''
planillas=pd.read_sql("SELECT entities.alias, ide.numero_documento, CONCAT(ide.nombre,' ', ide.apellido_paterno), tra.regimen_laboral, tra.remuneracion FROM payroll.ide INNER JOIN payroll.tra ON ide.cui=tra.cui_relacionado LEFT JOIN priv.entities ON entities.ruc=ide.ruc WHERE tra.situacion='ACTIVO' ORDER BY entities.alias", warehouse)
planillas


NameError: name 'pd' is not defined